# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [1]:
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

C:\Users\adobe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


How the game looks (without our agent):

In [3]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()

100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

$ \begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align} $

However, we don't know $ Q^* $. So, we use neural network as a approximators, we can simply create one and train it to resemble $ Q^* $.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

$ \begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align} $

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

$ \begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align} $

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [ ]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
        # initialise layers here
        self.dense_layer1 = nn.Linear(observation_size, observation_size)
        self.dense_layer2 = nn.Linear(observation_size, action_size)

    def forward(self, x):
        # send x through the network
        x = self.dense_layer1(x)
        x = self.dense_layer2(x)
        return x

    def train():
        


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [ ]:
import queue
import random

class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.model = Model(observation_size, action_size)
        self.memory = queue.Queue(100) # memory that stores N most new transitions
        # good place to store hyperparameters as attributes
        self.learning_rate = 0.7           # Learning rate
        self.gamma = 0.9                   # Discounting rate - Old value: 0.618

        self.total_episodes = 5000        # Total episodes
        # total_test_episodes = 5       # Total test episodes
        # max_steps = 99                # Max steps per episode

        # # Exploration parameters
        # epsilon = 1.0                 # Exploration rate
        # max_epsilon = 1.0             # Exploration probability at start
        # min_epsilon = 0.01            # Minimum exploration probability
        # decay_rate = 0.0001             # Exponential decay rate for exploration prob

    def remember(self, state, action, reward, next_state, done):
        # add to memory
        transition = [state, action, next_state, reward]
        self.memory.put(transition)

    def act(self, state):
        # return an action from the model
        self.model.forward(state)

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        for episode in self.total_episodes:
            state = env.reset()[0]
            for transition in batch_size:
                exp_exp_tradeoff = random.uniform(0, 1)

                ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
                if exp_exp_tradeoff > epsilon:
                    action = np.argmax(qtable[state, :])

                # Else doing a random choice --> exploration
                else:
                    action = env.action_space.sample()

                # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, truncated, info = env.step(action)


### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [ ]:
# def train(env, agent, episodes=1000, batch_size=64):  # train for many games
#     for _ in tqdm(range(episodes)):
#         state, _ = env.reset()
#         done = False
#         while not done:
#             # 1. make a move in game.
#             # 2. have the agent remember stuff.
#             # 3. update state
#             # 4. if we have enough experiences in out memory, learn from a batch with replay.
#             if len(agent.memory) >= batch_size:
#                 agent.replay(batch_size)
#     env.close()

### Putting it together
---
We train an agent on the environment:


In [ ]:
# env = gym.make('CartPole-v1')  # , render_mode='human')
# agent = Agent(env.observation_space.shape[0], env.action_space.n)
# train(env, agent)
# # torch.save(agent.model.state_dict(), 'model.pth')

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/